In [29]:
import pandas as pd
import pandas_ta as ta
import yfinance as yf

from invaas.schwab.cnn_fear_greed_index import get_historical_cnn_fear_greed_index


def get_df_fear_greed_index():
    df_fear_greed_index = pd.DataFrame(data=get_historical_cnn_fear_greed_index()["data"])
    df_fear_greed_index.set_index(
        pd.DatetimeIndex([pd.Timestamp(x, unit="ms", tz="UTC") for x in df_fear_greed_index.x]),
        inplace=True,
    )
    df_fear_greed_index.rename(columns={"y": "fear_greed_index"}, inplace=True)
    return df_fear_greed_index


def get_df_history(product_id: str, df_fear_greed_index: pd.DataFrame):
    df_history = yf.Ticker(product_id).history(interval="1d", period="1y")
    df_history.columns = map(str.lower, df_history.columns)

    timestamps = [pd.to_datetime(x, utc=True).round(freq="D") for x in df_history.index.values]
    timestamps_date_range = pd.date_range(start=timestamps[0], end=timestamps[-1], freq="D")
    df_history = df_history.set_index(pd.DatetimeIndex(timestamps)).reindex(timestamps_date_range, method="ffill")

    df_history = df_history.join(df_fear_greed_index[["fear_greed_index"]], how="inner")
    df_history["fear_greed_index"] = df_history.fear_greed_index.fillna(method="ffill").astype(float)
    df_history["fear_greed_index_max_14"] = df_history["fear_greed_index"].rolling(window=14, min_periods=14).max()

    RsiStrategy = ta.Strategy(name="RSI", ta=[{"kind": "rsi", "length": 14}])
    df_history.ta.strategy(RsiStrategy)
    df_history.sort_index(inplace=True)
    return df_history


df_fear_greed_index = get_df_fear_greed_index()
df_history = get_df_history("AAPL", get_df_fear_greed_index())

df_fear_greed_index
df_history.tail(50)

/var/folders/86/qt4jnsg167j7h7mqgbpp44kh0000gn/T/ipykernel_43268/608513667.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_history["fear_greed_index"] = df_history.fear_greed_index.fillna(method="ffill").astype(float)


,open,high,low,close,volume,dividends,stock splits,fear_greed_index,fear_greed_index_max_14,RSI_14
2023-11-10 00:00:00+00:00,183.970001,186.570007,183.529999,186.399994,66133400,0.24,0.0,41.457143,41.457143,69.295328
2023-11-13 00:00:00+00:00,185.820007,186.029999,184.210007,184.800003,43627500,0.00,0.0,39.885714,41.457143,65.068439
2023-11-14 00:00:00+00:00,187.699997,188.110001,186.300003,187.440002,60108400,0.00,0.0,46.600000,46.600000,68.484404
2023-11-15 00:00:00+00:00,187.850006,189.500000,187.779999,188.009995,53790500,0.00,0.0,52.542857,52.542857,69.185062
2023-11-16 00:00:00+00:00,189.570007,190.960007,188.649994,189.710007,54412900,0.00,0.0,55.057143,55.057143,71.238848
2023-11-17 00:00:00+00:00,190.250000,190.380005,188.570007,189.690002,50922700,0.00,0.0,58.857143,58.857143,71.178730
2023-11-20 00:00:00+00:00,189.889999,191.910004,189.880005,191.449997,46505100,0.00,0.0,61.285714,61.285714,73.312573
2023-11-21 00:00:00+00:00,191.410004,191.520004,189.740005,190.639999,38134500,0.00,0.0,61.171429,61.285714,70.717594
2023-11-22 00:00:00+00:00,191.490005,192.929993,190.830002,191.309998,39617700,0.00,0.0,65.057143,65.057143,71.612659
2023-11-24 00:00:00+00:00,190.869995,190.899994,189.250000,189.970001,24048300,0.00,0.0,66.857143,66.857143,67.189199


In [1]:
import json
import re
import requests

from playwright.async_api import async_playwright, Route, TimeoutError
from playwright_stealth import stealth_async
from requests.cookies import cookiejar_from_dict

from invaas.schwab.schwab_api import urls


# Constants
USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36"
VIEWPORT = {"width": 1920, "height": 1080}


class SessionManager:
    def __init__(self):
        self.headers = None
        self.session = requests.Session()
        
    async def setup(self):
        self.playwright = await async_playwright().start()
        self.browser = await self.playwright.firefox.launch(headless=True)
        self.page = await self.browser.new_page(user_agent=USER_AGENT, viewport=VIEWPORT)

        await stealth_async(self.page)
        
        async with self.page.expect_navigation():
            # self.page.on("request", lambda request: print(">>", request.method, request.url))
            # self.page.on("response", lambda response: print("<<", response.status, response.url))

            async def handle_page_response(response):
                if response.url == "https://en.macromicro.me/charts/data/50108":
                    self.historical_fear_greed_data = await response.json()
                    
            self.page.on("response", handle_page_response)
            
            await self.page.goto("https://en.macromicro.me/charts/50108/cnn-fear-and-greed")

        # await self.page.route(re.compile(r".*balancespositions*"), self.captureAuthToken)

    async def save_and_close_session(self):
        cookies = {cookie["name"]: cookie["value"] for cookie in await self.page.context.cookies()}
        self.session.cookies = cookiejar_from_dict(cookies)
        await self.page.close()
        await self.browser.close()
        await self.playwright.stop()

    async def get_historical_fear_greed_index
        

In [2]:
# %%sh
# playwright install

In [3]:
session_manager = SessionManager()
await session_manager.setup()
await session_manager.save_and_close_session()
session_manager.historical_fear_greed_data

{'success': 1,
 'data': {'c:50108': {'info': {'id': 50108,
    'slug': 'cnn-fear-and-greed',
    'type': 1,
    'name_tc': '美國-CNN恐懼與貪婪指數',
    'name_sc': '美国-CNN恐惧与贪婪指数',
    'name_en': 'US - CNN Fear and Greed Index',
    'country': '',
    'description_tc': 'CNN 恐懼與貪婪指數為採用美股市場情緒相關變數所編製的綜合指標，其中變數包括：市場動能、股價強度、股價廣度、 Put/Call Ratio 、市場波動、避險需求、垃圾債券需求。該指數計算這些單獨的變數偏離其平均值的程度，並賦予相同權重。其中 100 代表市場最貪婪，0 代表市場最恐懼。',
    'description_sc': 'CNN 恐惧与贪婪指数为采用美股市场情绪相关变数所编製的综合指标，其中变数包括：市场动能、股价强度、股价广度、 Put/Call Ratio 、市场波动、避险需求、垃圾债券需求。该指数计算这些单独的变数偏离其平均值的程度，并赋予相同权重。其中 100 代表市场最贪婪，0 代表市场最恐惧。',
    'description_en': 'The CNN Fear and Greed Index consists of the deviation from average of market momentum, stock price strength, stock price breadth, put and call options, market volatility, safe haven demand, and junk bond demand with equal weightings. \n \nThe index ranges from 0 to 100, with 100 representing maximum greed and 0 representing maximum fear.',
    'date_start': '2021-05-01',
    'date_end': None,
 